In [10]:
from decimal import Decimal, getcontext
import math

getcontext().prec = 30



class Vector:
    def __init__(self, coordinates):
        self.coordinates = coordinates
        self.dimension = len(coordinates)

    def __str__(self):
        return "Vector: {}".format(self.coordinates)
    
    def magnitude(self):
        return math.sqrt( sum( [x ** 2 for x in self.coordinates] ) )
    
    def direction(self):
        magnitude = self.magnitude()
        return tuple( x / magnitude for x in self.coordinates )
    
    def minus(self, v2):
        assert len( self.coordinates) == len( v2.coordinates )
        return Vector( tuple(coord - v2.coordinates[ix] for ix, coord in enumerate(self.coordinates) ))
    
    def dot_product( self, v2 ):
        assert len( self.coordinates) == len( v2.coordinates )
        s = 0
        for ix, coord in enumerate(self.coordinates):
            s += coord * v2.coordinates[ix]
        return s
    
    def angle( self, v2 ):
        dot = self.dot_product( v2 )
        cos_theta = dot / ( self.magnitude() * v2.magnitude() )
        return math.acos( cos_theta )
    
    def scalar_multiply(self, scalar):
        return Vector( tuple((scalar * c for c in self.coordinates)) )
    
    def parallel( self, v2 ):
        return ( self.direction() == v2.direction() or 
               self.direction() == -1 * v2.direction() )

    def orthogonal( self, v2 ):
        dot = self.dot_product( v2 )
        return abs(dot) < 0.001
    
    def projection( self, b ):
        magnitude = self.dot_product( b ) / b.magnitude()
        return Vector( b.direction() ).scalar_multiply( magnitude )
    
    def projection_ortho( self, b ):
        projection = self.projection( b )
        return self.minus(projection)

    def cross_product( self, b ):
        c1, c2 = self.coordinates, b.coordinates
        assert len(self.coordinates) == 3
        assert len(b.coordinates) == 3
        x = c1[1] * c2[2] - c1[2] * c2[1]
        y = c1[2] * c2[0] - c1[0] * c2[2]
        z = c1[0] * c2[1] - c1[1] * c2[0]
        
        return Vector( (x, y, z) )


class Line(object):

    NO_NONZERO_ELTS_FOUND_MSG = 'No nonzero elements found'

    def __init__(self, normal_vector=None, constant_term=None):
        self.dimension = 2

        if not normal_vector:
            all_zeros = ['0']*self.dimension
            normal_vector = Vector(all_zeros)
        self.normal_vector = normal_vector

        if not constant_term:
            constant_term = Decimal('0')
        self.constant_term = Decimal(constant_term)

        self.set_basepoint()


    def set_basepoint(self):
        try:
            n = self.normal_vector
            c = self.constant_term
            basepoint_coords = ['0']*self.dimension

            initial_index = Line.first_nonzero_index(n)
            initial_coefficient = n[initial_index]

            basepoint_coords[initial_index] = c/initial_coefficient
            self.basepoint = Vector(basepoint_coords)

        except Exception as e:
            if str(e) == Line.NO_NONZERO_ELTS_FOUND_MSG:
                self.basepoint = None
            else:
                print e
                e
                raise e


    def __str__(self):

        num_decimal_places = 3

        def write_coefficient(coefficient, is_initial_term=False):
            coefficient = round(coefficient, num_decimal_places)
            if coefficient % 1 == 0:
                coefficient = int(coefficient)

            output = ''

            if coefficient < 0:
                output += '-'
            if coefficient > 0 and not is_initial_term:
                output += '+'

            if not is_initial_term:
                output += ' '

            if abs(coefficient) != 1:
                output += '{}'.format(abs(coefficient))

            return output

        n = self.normal_vector

        try:
            initial_index = Line.first_nonzero_index(n)
            terms = [write_coefficient(n[i], is_initial_term=(i==initial_index)) + 'x_{}'.format(i+1)
                     for i in range(self.dimension) if round(n[i], num_decimal_places) != 0]
            output = ' '.join(terms)

        except Exception as e:
            if str(e) == self.NO_NONZERO_ELTS_FOUND_MSG:
                output = '0'
            else:
                raise e

        constant = round(self.constant_term, num_decimal_places)
        if constant % 1 == 0:
            constant = int(constant)
        output += ' = {}'.format(constant)

        return output


    @staticmethod
    def first_nonzero_index(iterable):
        for k, item in enumerate(iterable):
            if not MyDecimal(item).is_near_zero():
                return k
        raise Exception(Line.NO_NONZERO_ELTS_FOUND_MSG)

    def parallel(self, l2):
        return self.normal_vector.parallel(l2.normal_vector)
        

class MyDecimal(Decimal):
    def is_near_zero(self, eps=1e-10):
        return abs(self) < eps

In [11]:
l1 = Line(normal_vector=Vector( (4.046, 2.836) ), constant_term=1.21)
l2 = Line(normal_vector=Vector( (10.115, 7.09) ), constant_term=3.025)

l1.parallel(l2)

iteration over non-sequence


TypeError: iteration over non-sequence